In [ ]:
#1. If a decisoon tree is under-fitting the training dataset, is it a good idea to try scaling
#the input features?
    #Based on the notes, no. Scaling wouldn't matter because there is no need for 
    #scaling.
#2. If a decision tree is over-fitting the training dataset, is it a good idea to try decreasing
    #max depth?
    #Yes! Thsi will make the model less complex and help reularlize it!
#3. Why would you use a random forest instead of a decision tree?
    #d. a and b
        #a. For lower training error
        #b. To reduce the variance of the model
#4. Which of the following is/are TRUE about bagging trees?
    #d. a and c
        #a. In bagging trees, the trees are grown independent of each other.
        #c. Bagging is a method for improving the performance by aggregating the results of weak
        #learners.
#5. Picture attached!

In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.ensemble import RandomForestClassifier



s3= boto3.resource('s3')
bucket_name= 'morgangant-bata-445-bucket'
bucket= s3.Bucket(bucket_name)

file_key= 'framingham.csv'

bucket_object= bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the datefile
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [2]:
#Dropping na values
heart = heart.dropna()

In [40]:
importances = list()

for i in range (0,100):
    #Define the input and target variable
    x= heart[['male','age', 'education', 'currentSmoker', 'cigsPerDay', 'prevalentStroke', 'BPMeds', 'diabetes', 'totChol', 'prevalentHyp','BMI', 'heartRate', 'sysBP', 'diaBP', 'glucose']]
    y= heart['TenYearCHD']

    #Splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, stratify=y)

    #Building RF calssifier
    RF_md= RandomForestClassifier(n_estimators= 500).fit(x_train, y_train)

    #Extracting the feature importances
    importances.append(RF_md.feature_importances_)

In [41]:
a = pd.DataFrame(importances)
a.columns= ['male','age', 'education', 'currentSmoker', 'cigsPerDay', 'prevalentStroke', 'BPMeds', 'diabetes', 'totChol', 'prevalentHyp','BMI', 'heartRate', 'sysBP', 'diaBP', 'glucose']
a.apply(np.mean, axis = 0)

male               0.021190
age                0.124286
education          0.036895
currentSmoker      0.012531
cigsPerDay         0.050428
prevalentStroke    0.003248
BPMeds             0.007097
diabetes           0.006409
totChol            0.121908
prevalentHyp       0.018417
BMI                0.127188
heartRate          0.096220
sysBP              0.135234
diaBP              0.119373
glucose            0.119575
dtype: float64

In [3]:
md1_results= list()
md2_results= list()
md3_results= list()

#Define the input and target variable
x= heart[['age', 'BMI', 'sysBP', 'diaBP', 'glucose']]
y= heart['TenYearCHD']
    
for i in range (0,100):

    #splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, stratify=y)

    #Creating RF model with depth 3
    RF_md= RandomForestClassifier(max_depth= 3, n_estimators= 500).fit(x_train, y_train)
    #Predictingon the model
    RF_pred= RF_md.predict_proba(x_test)[:,1]
    RF_pred
    #Changling likleyhoods to labels
    RF_label= np.where(RF_pred < .1, 0, 1)
    #Computing recall
    Model1_recall= recall_score(y_test, RF_label)
    md1_results.append(Model1_recall)


    #Creating RF model with depth 5
    RF_md2= RandomForestClassifier(max_depth= 5, n_estimators= 500).fit(x_train, y_train)
    #Predictingon the model
    RF_pred2= RF_md2.predict_proba(x_test)[:,1]
    RF_pred2
    #Changling likleyhoods to labels
    RF_label2= np.where(RF_pred2 < .1, 0, 1)
    #Computing recall
    Model2_recall= recall_score(y_test, RF_label2)
    md2_results.append(Model2_recall)


    #Creating RF model with depth 5
    RF_md3= RandomForestClassifier(max_depth= 7, n_estimators= 500).fit(x_train, y_train)
    #Predictingon the model
    RF_pred3= RF_md3.predict_proba(x_test)[:,1]
    RF_pred3
    #Changling likleyhoods to labels
    RF_label3= np.where(RF_pred3 < .1, 0, 1)
    #Computing recall
    Model3_recall= recall_score(y_test, RF_label3)
    md3_results.append(Model3_recall)

In [4]:
print('Average Recall for model 1:', np.mean(md1_results))
print('Average Recall for model 2:',np.mean(md2_results))
print('Average Recall for model 3:',np.mean(md3_results))

Average Recall for model 1: 0.8418750000000002
Average Recall for model 2: 0.8208035714285714
Average Recall for model 3: 0.8041071428571428


In [ ]:
#Based on the averages above, I would use model 1 (500 trees with a depth of 3) because on average, it had the highets recall!!